# STAR PERFORMERS

In [59]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
import xmltodict
from pprint import pprint
import os
import csv
from project_titan/api_keys import z_keys

SyntaxError: invalid syntax (<ipython-input-59-4061093456f2>, line 12)

In [2]:
ny18_df = pd.read_csv("Files/2018NYSales.csv")

In [3]:
len(ny18_df)

16369

In [4]:
ny18_df = ny18_df.rename(columns={' SALE PRICE ': 'SALE PRICE'})

In [5]:
ny18_df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE'],
      dtype='object')

In [6]:
ny18_df.describe()

,BOROUGH,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,YEAR BUILT,TAX CLASS AT TIME OF SALE
count,16369.0,16369.000000,16369.000000,16369.000000,16369.000000,16369.000000,16369.000000,16369.000000,16369.000000
mean,1.0,1134.104160,736.368990,10015.058281,1.837681,0.341438,2.305761,1702.245586,2.120716
std,0.0,526.607997,910.316535,385.473717,13.408085,4.091756,14.379706,652.533099,0.541500
min,1.0,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1.0,798.000000,29.000000,10013.000000,0.000000,0.000000,0.000000,1910.000000,2.000000
50%,1.0,1171.000000,1002.000000,10022.000000,0.000000,0.000000,1.000000,1931.000000,2.000000
75%,1.0,1469.000000,1185.000000,10028.000000,1.000000,0.000000,1.000000,1973.000000,2.000000
max,1.0,2250.000000,9101.000000,10463.000000,597.000000,180.000000,601.000000,2018.000000,4.000000


In [7]:
ny18_df["ADDRESS"] = ny18_df["ADDRESS"].str.strip()
ny18_df['ADDRESS'] = ny18_df['ADDRESS'].str.split(' ').apply(lambda x: '+'.join(x))
ny18_df.count()

BOROUGH                           16369
NEIGHBORHOOD                      16369
BUILDING CLASS CATEGORY           16369
TAX CLASS AT PRESENT              16369
BLOCK                             16369
LOT                               16369
EASE-MENT                         16369
BUILDING CLASS AT PRESENT         16369
ADDRESS                           16369
APARTMENT NUMBER                  16369
ZIP CODE                          16369
RESIDENTIAL UNITS                 16369
COMMERCIAL UNITS                  16369
TOTAL UNITS                       16369
LAND SQUARE FEET                  16369
GROSS SQUARE FEET                 16369
YEAR BUILT                        16369
TAX CLASS AT TIME OF SALE         16369
BUILDING CLASS AT TIME OF SALE    16369
SALE PRICE                        16369
SALE DATE                         16369
dtype: int64

In [8]:
ny18_df["SALE PRICE"] = ny18_df.loc[:,"SALE PRICE"].replace('[\$,]', '', regex=True)
ny18_df["SALE PRICE"] = ny18_df.loc[:,"SALE PRICE"].replace('[-,]', '0', regex=True)

In [9]:
# Removing home prices less than 100K to avoid a bias after preliminary research on what these lower sale prices rep.
# like: Internal family sale transfer at $0, garage sale etc.

ny18_df = ny18_df[ny18_df['SALE PRICE'].astype(float) > 100000]
ny18_df = ny18_df.drop_duplicates(subset=['ADDRESS'], keep = 'first')

In [10]:
len(ny18_df)

8951

In [11]:
ny18_df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,390,61,,A4,189+EAST+7TH+STREET,,...,1,0,1,987,"2,183",1860,1,A4,4844809,5/22/2018
2,1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,400,19,,A4,526+EAST+5TH+STREET,,...,1,0,1,"1,883","5,200",1900,1,A4,6100000,12/3/2018
6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,379,37,,C7,133+AVENUE+D,,...,20,2,22,"3,481","14,588",1930,2,C7,8300000,12/10/2018
7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,385,2,,C7,21-23+AVENUE+B,,...,12,4,16,"4,186","10,588",1900,2,C7,10350000,9/7/2018
8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,389,18,,C1,200+EAST+7TH+STREET,,...,18,0,18,"2,271","10,650",1910,2,C1,5000000,9/5/2018


In [12]:
response = requests.get(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address=635+west+42+street&citystatezip=10036&rentzestimate=true")

In [13]:
print(response)

<Response [200]>


In [14]:
#Using BeautifulSoup to convert XML text to pretty format
soup = BeautifulSoup(response.text)
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<html>
 <body>
  <searchresults:searchresults xmlns:searchresults="http://www.zillow.com/static/xsd/SearchResults.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemalocation="http://www.zillow.com/static/xsd/SearchResults.xsd https://www.zillowstatic.com/vstatic/091b98a/static/xsd/SearchResults.xsd">
   <request>
    <address>
     635 west 42 street
    </address>
    <citystatezip>
     10036
    </citystatezip>
   </request>
   <message>
    <text>
     Request successfully processed
    </text>
    <code>
     0
    </code>
   </message>
   <response>
    <results>
     <result>
      <zpid>
       2129887786
      </zpid>
      <links>
       <homedetails>
        https://www.zillow.com/homedetails/635-W-42nd-Street-42a-42A-New-York-NY-10036/2129887786_zpid/
       </homedetails>
       <mapthishome>
        http://www.zillow.com/homes/2129887786_zpid/
       </mapthishome>
       <comparables>
        http://www.zillow.com/

In [ ]:
doc = xmltodict.parse(response.text)['SearchResults:searchresults']['response']['results']['result']
len(doc)

In [ ]:
doc[1]['address']['street']

In [46]:
def need_rent_estimate(address):
    return (address.get('rentzestimate')) and not(address.get('lastSoldPrice'))

In [47]:
def need_sale_estimate(address):
    return (not(address.get('rentzestimate')) and (address.get('lastSoldPrice')))

In [43]:
def get_rent_estimate(address):
    print("Entering get_rent_estimate")
    rent = address['rentzestimate']['amount']['#text']
    address = address['address']['street']
#     zipcode = address['address']['zipcode']
    var = 2
#     return pd.Series({'rent':rent, 'address':address,'zipcode':zipcode,'var':var})
    return pd.Series({'rent':rent, 'address':address,'var':var})

In [44]:
def get_property_estimate(address):
    print("Entering get_property_estimate")
    sold = address['lastSoldPrice']['#text']
    sold_date = address['lastSoldDate']
#     zestimate=address['zestimate']['amount']['#text']
    sqft = address['finishedSqFt']
    address = address['address']['street']
#     zipcode = address['address']['zipcode']
    var = 1
    return pd.Series({'sold':sold, 'var':var}) 

In [ ]:
ny18_df = ny18_df.reset_index()
ny18_df.head()

In [ ]:
ny18_df.loc[0,'BLOCK']

In [57]:
block =[]
lot=[]
ny_sale_amt =[]
ny_sale_date=[]
series = []
curdoc={}

for address, zipcode in zip(ny18_df['ADDRESS'],ny18_df['ZIP CODE']):
    try:
       i = 0
       print(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address={address}&citystatezip={zipcode}&rentzestimate=true")
       response_main = requests.get(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address={address}&citystatezip={zipcode}&rentzestimate=true")
#        block=ny18_Update['BLOCK']
#        lot=ny18_Update['LOT']
#        sale_amt=ny18_Update['SALE PRICE']
#        sale_date=ny18_Update['SALE DATE']
       doc = xmltodict.parse(response_main.text, force_list={'result': True})['SearchResults:searchresults']['response']['results']['result']

       while i < len(doc):
           print(f"{doc[i]['address']['street']} \n")

           if need_rent_estimate(doc[i]):
               new_series = get_rent_estimate(doc[i])
           elif need_sale_estimate(doc[i]):
               new_series = get_property_estimate(doc[i])
           
           i+=1
           print(f'End of record {i}')
           print('-------------------------------')
           series.append(new_series)
       time.sleep(30)
       df = pd.concat(series, 1,sort=True).T
    except KeyError:
       print(f"Only 1 record: {doc[i]['address']['street']} \n")

https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz1gxoufbrk0b_1pt0f&address=189+EAST+7TH+STREET&citystatezip=10009&rentzestimate=true
189 E 7th St 

Entering get_property_estimate
End of record 1
-------------------------------
189 E 7th St 

End of record 2
-------------------------------
189 E 7th St 

Entering get_rent_estimate
End of record 3
-------------------------------
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz1gxoufbrk0b_1pt0f&address=526+EAST+5TH+STREET&citystatezip=10009&rentzestimate=true
526 E 5th St 

End of record 1
-------------------------------
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz1gxoufbrk0b_1pt0f&address=133+AVENUE+D&citystatezip=10009&rentzestimate=true
133 Avenue D APT 16 

Entering get_rent_estimate
End of record 1
-------------------------------
133 Avenue D APT 11 

Entering get_rent_estimate
End of record 2
-------------------------------
133 Avenue D APT 17 

Entering get_

KeyboardInterrupt: 

In [58]:
# xmltodict.parse(response_main.text)['SearchResults:searchresults'].keys()

odict_keys(['@xsi:schemaLocation', '@xmlns:xsi', '@xmlns:SearchResults', 'request', 'message', 'response'])